# HU Extension       Assignment 12             E63 Big Data Analytics 		                  	
## Handed out: 04/21/2023                           Due by 11:59 PM EST on Saturday, 04/29/2023
### Emmanuel Aboah

### Problem 1.

Install delta-spark Python package. Confirm that your installation is valid by creating a table in the local file system. Report on the location of the files (directories) containing your data. How many parquet files you created with the first save operation. Demonstrate that you can read saved data.

In [79]:
# Imports
import findspark
from pyspark import SparkContext
from pyspark import SparkContext
from pyspark.sql import SparkSession

findspark.init()

"""
    Import jars from maven central.
    Follows Format -> groupId:artifactId:version
"""
packages = ["io.delta:delta-core_2.12:2.3.0"]

# Session Builder
spark = (
    SparkSession.builder.master("local[*]")
    .appName("delta_lakes_app")
    .config("spark.jars.packages", ",".join(packages))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .getOrCreate()
)

sc = SparkContext.getOrCreate()

sc.setLogLevel("ERROR")

Create in local file system

In [80]:
# Create test data

test_data = spark.range(0, 15)

test_data.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



In [81]:
OUTPUT_PATH = "output_data/test-table"

In [82]:
# Save to filesystem
(
    test_data.write
    .format("delta")
    .mode("overwrite")
    .save(OUTPUT_PATH)
)

Report on the location of the files (directories) containing your data

In [83]:
ls output_data/test-table -la

total 76
drwxr-xr-x 3 manny manny 4096 Apr 28 18:51 ./
drwxr-xr-x 3 manny manny 4096 Apr 28 18:51 ../
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00000-c8b93c83-d679-4371-9bd0-74fc258443cf-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00001-b21482c7-0789-41e3-b162-86f6f4ac63b4-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00002-464b4fb0-65bb-4300-b6a5-757bbdc7ea01-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00003-8c9de733-d20d-45b0-ad4f-a75d9d75627a-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00004-7ba3280b-9cfa-46f2-837c-cbaa7c082b3f-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00005-d367405b-684e-489e-a26c-0451434cb4ab-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00006-66c8138a-abfa-4ff8-a6e2-655e538071e7-c000.snappy.parquet.crc
-rw-r--r-- 1 manny manny   12 Apr 28 18:51 .part-00007-8bf98dc1-b13f-42a5-86c6-fedce66

In [84]:
ls -1A output_data/test-table | wc -l

17


Demonstrate that you can read saved data.

In [85]:
test_data_df = (
    spark.read
    .format("delta")
    .load(OUTPUT_PATH)
)

test_data_df.show()

+---+
| id|
+---+
|  1|
|  2|
| 11|
| 12|
| 13|
| 14|
|  5|
|  6|
|  3|
|  4|
|  9|
| 10|
|  7|
|  8|
|  0|
+---+



### Problem 2. 

Go through 4 successive operations: one update, one insert, one merge and one delete of the data in your delta table created in Problem 1. Use time travel to demonstrate that you see the state of the table before each operation. 

In [86]:
from delta.tables import DeltaTable
from pyspark.sql.functions import expr, col

# Create a Delta Table
delta_table = DeltaTable.forPath(spark, OUTPUT_PATH)

In [87]:
# Update table by multiplying each value by 2
delta_table.update(set={"id": expr("id * 2")})

In [88]:
upd_df = (
    spark.read
    .format("delta")
    .load(OUTPUT_PATH)
)

upd_df.show()

+---+
| id|
+---+
|  6|
|  8|
| 26|
| 28|
| 18|
| 20|
| 10|
| 12|
| 22|
| 24|
| 14|
| 16|
|  2|
|  4|
|  0|
+---+



In [89]:
# Upsert/ merge the numbers 16 - 20
new_data = spark.range(16, 21)

(
    delta_table.alias("prev_data")
    .merge(
        new_data.alias("new_data"),
        "prev_data.id = new_data.id")
    .whenMatchedUpdate(set= {"id": col("new_data.id")})
    .whenNotMatchedInsert(values= {"id": col("new_data.id")})
    .execute()
)

delta_table.toDF().show()

+---+
| id|
+---+
| 14|
| 16|
| 17|
| 18|
| 19|
| 20|
|  6|
|  8|
| 26|
| 28|
| 10|
| 12|
| 22|
| 24|
|  2|
|  4|
|  0|
+---+



In [90]:
# Delete records > 19
delta_table.delete(condition=expr("id > 19"))

delta_table.toDF().show()

+---+
| id|
+---+
| 14|
| 16|
| 17|
| 18|
| 19|
|  6|
|  8|
| 10|
| 12|
|  2|
|  4|
|  0|
+---+



In [91]:
# Use time travel to show different version states

# version 1
df1 = (
    spark.read
    .format("delta")
    .option("versionAsOf", 0)
    .load(OUTPUT_PATH)
)

df1.orderBy("id").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



In [92]:
# version 2
df2 = (
    spark.read
    .format("delta")
    .option("versionAsOf", 1)
    .load(OUTPUT_PATH)
)

df2.orderBy("id").show()

+---+
| id|
+---+
|  0|
|  2|
|  4|
|  6|
|  8|
| 10|
| 12|
| 14|
| 16|
| 18|
| 20|
| 22|
| 24|
| 26|
| 28|
+---+



In [93]:
# version 3
df1 = (
    spark.read
    .format("delta")
    .option("versionAsOf", 2)
    .load(OUTPUT_PATH)
)

df1.orderBy("id").show()

+---+
| id|
+---+
|  0|
|  2|
|  4|
|  6|
|  8|
| 10|
| 12|
| 14|
| 16|
| 17|
| 18|
| 19|
| 20|
| 22|
| 24|
| 26|
| 28|
+---+



In [95]:
# version 4
df1 = (
    spark.read
    .format("delta")
    .option("versionAsOf", 3)
    .load(OUTPUT_PATH)
)

df1.orderBy("id").show()

+---+
| id|
+---+
|  0|
|  2|
|  4|
|  6|
|  8|
| 10|
| 12|
| 14|
| 16|
| 17|
| 18|
| 19|
+---+



### Problem 3. 

Create a standalone Python script that will perform a conditional uperst on your table in Problem 2. Let the script query and print the content of the table.

```py
"""🐍 __python__ script to perform upsert as per problem 3
"""
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta import configure_spark_with_delta_pip, DeltaTable

findspark.init()

# Output directory
OUTPUT_PATH = "output_data/test-table"

# Create a builder with the Delta extensions
builder = (
    SparkSession.builder.appName("delta_lake_app")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

# Create a Spark instance with the builder
# As a result, we now can read and write Delta files
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Create a Delta Table
delta_table = DeltaTable.forPath(spark, OUTPUT_PATH)

# Show table prior to upsert
print("Test Delta Table prior to upsert")
delta_table.toDF().show()

# Upsert/ merge the numbers 16 - 20
new_data = spark.range(20, 41)

(
    delta_table.alias("prev_data")
    .merge(new_data.alias("new_data"), "prev_data.id = new_data.id")
    .whenMatchedUpdate(set={"id": col("new_data.id")})
    .whenNotMatchedInsert(values={"id": col("new_data.id")})
    .execute()
)

# Show updated data
print("Updated Delta Table")

delta_table.toDF().show()
```

Run the python script

```bash
python delta_lake_upserts.py
```

Output

```bash
(BDA2023) manny@LAPTOP-85L1BUVJ:~/dev/cscie-63/hw12_delta_lakes$ python delta_lake_upserts.py
23/04/28 18:52:58 WARN Utils: Your hostname, LAPTOP-85L1BUVJ resolves to a loopback address: 127.0.1.1; using 172.19.201.226 instead (on interface eth0)
23/04/28 18:52:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/manny/dev/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/manny/.ivy2/cache
The jars for the packages stored in: /home/manny/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9cb11ddc-5aa3-4205-a825-ed38e2e85a69;1.0
        confs: [default]
        found io.delta#delta-core_2.12;2.3.0 in central
        found io.delta#delta-storage;2.3.0 in central
        found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 468ms :: artifacts dl 22ms
        :: modules in use:
        io.delta#delta-core_2.12;2.3.0 from central in [default]
        io.delta#delta-storage;2.3.0 from central in [default]
        org.antlr#antlr4-runtime;4.8 from central in [default]
        ---------------------------------------------------------------------
        |                  |            modules            ||   artifacts   |
        |       conf       | number| search|dwnlded|evicted|| number|dwnlded|
        ---------------------------------------------------------------------
        |      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
        ---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-9cb11ddc-5aa3-4205-a825-ed38e2e85a69
        confs: [default]
        0 artifacts copied, 3 already retrieved (0kB/22ms)
23/04/28 18:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/28 18:53:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Test Delta Table prior to upsert                                                
23/04/28 18:53:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+---+                                                                           
| id|
+---+
| 14|
| 16|
| 17|
| 18|
| 19|
|  6|
|  8|
| 10|
| 12|
|  2|
|  4|
|  0|
+---+

23/04/28 18:53:39 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
23/04/28 18:53:40 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
Updated Delta Table                                                             
+---+                                                                           
| id|
+---+
| 21|
| 20|
| 22|
| 23|
| 24|
| 26|
| 25|
| 29|
| 27|
| 28|
| 32|
| 31|
| 30|
| 34|
| 33|
| 37|
| 35|
| 36|
| 39|
| 38|
+---+
only showing top 20 rows
```

### Problem 4. 

Search through delta.io documentation to find the way to describe your table. 

In [102]:
# describe table

delta_table.detail().show()

+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
|format|                  id|name|description|            location|           createdAt|        lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|       tableFeatures|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+-----------+----------+----------------+----------------+--------------------+
| delta|f6fbe204-b060-4a5...|null|       null|file:/home/manny/...|2023-04-28 18:51:...|2023-04-28 18:53:...|              []|       6|       3011|        {}|               1|               2|[appendOnly, inva...|
+------+--------------------+----+-----------+--------------------+--------------------+--------------------+----------------+--------+---------

### Problem 5. 

Search through delta.io documentation to find the way to print the history of your table. Present the timestamp when the operation was performed and the type of the operation performed.

In [105]:
history_df = delta_table.history()

history_df.select("timestamp", "operation").show()

+--------------------+---------+
|           timestamp|operation|
+--------------------+---------+
|2023-04-28 18:53:...|    MERGE|
|2023-04-28 18:52:...|   DELETE|
|2023-04-28 18:52:...|    MERGE|
|2023-04-28 18:51:...|   UPDATE|
|2023-04-28 18:51:...|    WRITE|
+--------------------+---------+

